In [1]:
import pandas as pd
import numpy as np

In [87]:
kadaster = pd.read_csv("kadaster/kadaster_merged_dataset_corrected.csv", dtype=str)
sbi = pd.read_csv("kvk/sbi_codes/kvk_sbi_codes.csv", dtype=object)
kvk = pd.read_excel("kvk/DatasetKvK01102019.xls", dtype=object)
niwo = pd.read_csv("niwo/niwo_dict.csv", dtype=object)
fenex = pd.read_csv("fenex/fenex.csv", dtype=object)
company_info = pd.read_csv("company_info/kvk_companyinfo.csv", dtype=object)

In [88]:
kvk_drop = ['RGL','STRVA', 'PCPLVA', 'STRCA', 'PCPLCA','HN1X30', 'HN1X2X30', 'HN2X2X30', \
            'PCCA', 'GEMK_VA', 'GEMK_CA', 'GEMNAAM', 'PROV', 'STRAAT_VA', \
            'TOEV_VA', 'PC_VA', 'PLAATS_VA', 'STRAAT_CA', 'HUISNR_CA', \
            'TOEV_CA', 'PC_CA', 'PLAATS_CA', 'NMI', 'HFD_N_VEST', 'C_HOOFDACT', 'VERK_ACT', \
            'NEVEN1', 'NEVEN2', 'IND_IM_EXP', 'URL', 'BEHKN', 'GEOKN']
kadaster_drop = ['nrAanduiding','vblObject']

In [89]:
kvk_cluster = kvk.drop(kvk_drop, axis=1)
kadaster_cluster = kadaster.drop(kadaster_drop, axis=1)

In [90]:
kvk_cluster["HUISNR_VA"] = kvk_cluster["HUISNR_VA"].astype('int64')
kadaster_cluster["huisnummer"] = kadaster_cluster["huisnummer"].astype('int64')

In [91]:
kvk_kadaster = pd.merge(left=kvk_cluster, right=kadaster_cluster, how='left', left_on=["PCVA","HUISNR_VA"], \
                        right_on=["postcode","huisnummer"])
kvk_kad_sbi = pd.merge(left=kvk_kadaster, right=sbi, how='left', on=["DOSSIER","VGNUMMER"])
kvk_kad_sbi_comp = pd.merge(left=kvk_kad_sbi, right=company_info, how='left', left_on="DOSSIER", right_on="kvknr")

In [92]:
niwo_kvknr = niwo["kvk_nummer"].to_list()
fenex_compname = [x.lower().strip() for x in fenex["Company Name"].to_list()]

In [93]:
kvk_kad_sbi_comp['inNiwo'] = [1 if x in niwo_kvknr else 0 for x in kvk_kad_sbi_comp['DOSSIER']] 
kvk_kad_sbi_comp["inFenex"] = [1 if x.lower() in fenex_compname else 0 for x in kvk_kad_sbi_comp["HN45"].astype(str)]

In [94]:
kvk_kad_sbi_comp.head()

,DOSSIER,VGNUMMER,HN45,PCVA,HUISNR_VA,RV,huisnummer,postcode,inOnderzoek,gebruiksdoel,oppervlakte,vbloStatus,C_HOOFDACT,NEVEN1,NEVEN2,kvknr,reg_date,emp_class,inNiwo,inFenex
0,01016460,000018080715,Falco Lines B.V.,2984BG,9,41,9.0,2984BG,False,kantoorfunctie,214,in gebruik,H,NaN,NaN,01016460,21-05-2001,05,0,0
1,01036591,000015881202,Distributie en Container Centrum B.V.,8911AW,1,41,1.0,8911AW,False,kantoorfunctie,394,in gebruik,H,N,N,01036591,14-12-1972,01,0,0
2,01038431,000015998142,Pax Bouw en Industrieservice B.V.,8448CN,2,41,2.0,8448CN,False,industriefunctie,5154,in gebruik,H,C,H,01038431,14-12-1972,06,1,0
3,01049319,000017297222,Water-Land Bevrachtingen B.V.,8801JK,3,41,3.0,8801JK,False,kantoorfunctie,278,in gebruik,H,NaN,NaN,01049319,02-05-1983,04,1,0
4,01056343,000017633524,"Koel-, Vries- en Opslagbedrijf Gosliga B.V.",9061AE,12,41,12.0,9061AE,False,overige gebruiksfunctie,97,in gebruik,H,H,H,01056343,23-03-1988,01,0,0


In [95]:
drop_cols = ["HUISNR_VA","PCVA","huisnummer","postcode","kvknr","DOSSIER","HN45"]
kvk_kad_sbi_comp = kvk_kad_sbi_comp.drop(drop_cols,axis=1)

In [96]:
cluster_data = kvk_kad_sbi_comp

In [97]:
def onehot_encoder(df, prefix):
    one_hot = pd.get_dummies(df[prefix], prefix=prefix)
    df = df.drop([prefix], axis=1)
    df = df.join(one_hot)
    return df

In [98]:
cluster_data["RV"] = cluster_data["RV"].astype(int)
cluster_data["oppervlakte"] = cluster_data["oppervlakte"].astype(float)
cluster_data["emp_class"] = cluster_data["emp_class"].replace(to_replace=[np.nan,'io'],value=None)
cluster_data["emp_class"] = cluster_data["emp_class"].astype(int)
cluster_data["reg_date"] = cluster_data["reg_date"].replace(to_replace=['ull,"emplo','(w,d,s,l,i'], value=None)
cluster_data['reg_date'] = pd.to_datetime(cluster_data['reg_date'])

In [99]:
cluster_data = onehot_encoder(cluster_data,"gebruiksdoel")
cluster_data = onehot_encoder(cluster_data,"vbloStatus")
cluster_data = onehot_encoder(cluster_data,"C_HOOFDACT")
cluster_data = onehot_encoder(cluster_data, "NEVEN1")
cluster_data = onehot_encoder(cluster_data, "NEVEN2")
cluster_data = onehot_encoder(cluster_data, "RV")

In [100]:
cluster_data.head()

,VGNUMMER,inOnderzoek,oppervlakte,reg_date,emp_class,inNiwo,inFenex,gebruiksdoel_bijeenkomstfunctie,gebruiksdoel_celfunctie,gebruiksdoel_gezondheidszorgfunctie,...,RV_40,RV_41,RV_42,RV_51,RV_61,RV_71,RV_74,RV_93,RV_94,RV_96
0,000018080715,False,214.0,2001-05-21,5,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,000015881202,False,394.0,1972-12-14,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,000015998142,False,5154.0,1972-12-14,6,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,000017297222,False,278.0,1983-02-05,4,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,000017633524,False,97.0,1988-03-23,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [101]:
cluster_data.dtypes

VGNUMMER                                          object
inOnderzoek                                       object
oppervlakte                                      float64
reg_date                                  datetime64[ns]
emp_class                                          int64
inNiwo                                             int64
inFenex                                            int64
gebruiksdoel_bijeenkomstfunctie                    uint8
gebruiksdoel_celfunctie                            uint8
gebruiksdoel_gezondheidszorgfunctie                uint8
gebruiksdoel_industriefunctie                      uint8
gebruiksdoel_kantoorfunctie                        uint8
gebruiksdoel_logiesfunctie                         uint8
gebruiksdoel_onderwijsfunctie                      uint8
gebruiksdoel_overige gebruiksfunctie               uint8
gebruiksdoel_sportfunctie                          uint8
gebruiksdoel_winkelfunctie                         uint8
gebruiksdoel_woonfunctie       

In [ ]:
print(__doc__)

import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()